<a href="https://colab.research.google.com/github/Vasamsetti-venkat/FMML_LAB5/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Yes, averaging the validation accuracy across multiple splits of your dataset, often referred to as cross-validation, can provide more consistent and reliable results when assessing the performance of a machine learning model. Here's why it helps with consistency:

1. **Reduces Variance**: When you split your dataset into multiple subsets (folds) and train/validate your model on each of them, you get a better sense of how your model performs on different data samples. This helps to reduce the variance in your evaluation, as you're not relying on a single random split.

2. **Better Generalization Estimate**: Cross-validation provides a more robust estimate of how well your model generalizes to unseen data. By evaluating the model on multiple different subsets, you gain confidence that the performance is representative of the model's true capabilities.

3. **Effective Use of Data**: It allows you to make efficient use of your data, especially when you have limited samples. In k-fold cross-validation, for example, each data point is used for both training and validation, maximizing data utilization.

4. **Detects Overfitting**: Cross-validation can help you detect overfitting. If a model consistently performs well on one subset but poorly on others, it might be overfitting to the training data.

Common techniques include k-fold cross-validation, stratified k-fold cross-validation, and leave-one-out cross-validation, among others. The choice of the specific cross-validation technique depends on your dataset size and characteristics.

In summary, averaging validation accuracy across multiple splits of your data is a standard practice in machine learning to provide more consistent and reliable performance estimates for your models.
2. Does it give more accurate estimate of test accuracy?

Cross-validation, which involves averaging validation accuracy across multiple splits of your dataset, does not directly provide a more accurate estimate of test accuracy. Instead, it provides a more reliable estimate of how well your model is likely to perform on new, unseen data. Here's why:

1. **Validation vs. Test Data**: In machine learning, there is a distinction between validation data and test data.
   
   - Validation data is used during model training to tune hyperparameters and assess model performance. It is typically a subset of your training data.
   - Test data is completely held out and not used during training or hyperparameter tuning. It is used to evaluate your model's generalization to new, unseen data.

2. **Cross-Validation Purpose**: Cross-validation helps you estimate how well your model might generalize to new data by simulating the process of training and validating on different subsets of your training data. It helps assess the model's performance consistency and whether it's likely to perform well on unseen data.

3. **Test Accuracy**: Test accuracy is the most accurate estimate of how well your model performs on unseen data because it uses data that the model has never seen before. It provides a realistic assessment of your model's generalization.

4. **Validation as an Indicator**: While cross-validation doesn't directly estimate test accuracy, it can be considered an indicator of how well your model might perform on unseen data. If your model performs well on validation data across multiple splits (consistent results), it suggests that the model might generalize well to unseen data. However, it doesn't replace the need for a final test on completely independent test data.

In summary, cross-validation is a valuable technique for assessing the reliability and consistency of your model's performance. It can provide insights into how well your model generalizes, but the most accurate estimate of test accuracy comes from evaluating your model on a dedicated test dataset that it has never seen before.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

In the context of machine learning, the number of iterations can have an impact on the estimate of a model's performance, but it doesn't necessarily lead to a better estimate with higher iterations. The relationship between the number of iterations and the estimate of performance depends on several factors:

1. **Convergence**: For iterative optimization algorithms, such as gradient descent in training neural networks, increasing the number of iterations can help the model converge to a better solution. Convergence refers to the process where the model's parameters (weights) gradually reach values that minimize the loss function. In this sense, more iterations can lead to better training performance.

2. **Overfitting**: However, there's a risk associated with excessive iterations, especially in complex models. If you train for too many iterations, the model may start to overfit the training data, which means it becomes very specialized in fitting the training data but generalizes poorly to new, unseen data. This can result in a high training performance but a poor test performance.

3. **Computational Cost**: Increasing the number of iterations can significantly increase the computational cost of training a model. Training deep learning models, for example, can be computationally expensive, and a balance needs to be struck between training time and model performance.

4. **Early Stopping**: To address the trade-off between convergence and overfitting, practitioners often use techniques like early stopping. Early stopping involves monitoring the model's performance on a validation set during training and stopping when the performance starts to degrade, indicating overfitting. This helps prevent excessive iterations.

5. **Hyperparameter Tuning**: The number of iterations can be considered a hyperparameter that needs to be tuned along with other hyperparameters like learning rate, batch size, etc. The optimal number of iterations can vary from one problem and dataset to another.

In summary, the effect of the number of iterations on the estimate of a model's performance is complex. More iterations can help the model converge, but they can also lead to overfitting and increased computational cost. It's essential to strike a balance and potentially use techniques like early stopping and hyperparameter tuning to determine the optimal number of iterations for your specific problem.

4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Increasing the number of iterations can help to some extent when dealing with very small training or validation datasets, but it is not a complete solution to the challenges posed by small datasets. Here are some considerations:

1. **Overfitting**: When you have a very small training dataset, increasing the number of iterations can make your model fit the training data more closely. While this might reduce training error, it can also lead to overfitting. Overfitting occurs when the model becomes too specific to the training data and does not generalize well to new, unseen data.

2. **Limited Diversity**: Small datasets often lack the diversity and variability necessary for training a robust model. Even with more iterations, the model may not capture the full range of patterns and variations present in the data.

3. **Validation Data**: Increasing iterations can also lead to overfitting on the validation data if it's small. When you have a limited validation dataset, the model may start to perform well on that specific subset but poorly on new, unseen data.

4. **Early Stopping**: It's important to use techniques like early stopping and regularization when working with small datasets. Early stopping can help prevent overfitting by monitoring the model's performance on a validation set and stopping training when performance degrades.

5. **Data Augmentation**: Data augmentation techniques can be particularly useful for small datasets. These techniques artificially increase the effective size of your dataset by applying transformations to the existing data (e.g., rotations, flips, noise addition). This can help the model generalize better.

6. **Transfer Learning**: If your dataset is extremely small, you might consider using transfer learning, where you fine-tune a pre-trained model on your small dataset. Transfer learning leverages knowledge learned from a larger dataset and adapts it to your specific task.

In conclusion, while increasing the number of iterations may have some positive effects, it's essential to be cautious when dealing with very small datasets. Focus on preventing overfitting, using regularization techniques, considering data augmentation, and exploring transfer learning if applicable. The small dataset's inherent limitations may require creative solutions beyond simply increasing iterations.